In [1]:

import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

import scipy
from scipy.spatial.distance import cdist
from ncuts_utils import ncuts_chunk,kDTree_1NN_feature_reprojection_colors, get_merge_pcds
from dataset_utils import * 
from point_cloud_utils import get_pcd, transform_pcd, kDTree_1NN_feature_reprojection, remove_isolated_points, get_subpcd, get_statistical_inlier_indices, merge_chunks_unite_instances, merge_chunks_unite_instances2, remove_semantics, get_merge_pcds, merge_unite_gt
from aggregate_pointcloud import aggregate_pointcloud
from visualization_utils import generate_random_colors, color_pcd_by_labels,generate_random_colors_map
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch, dinov2_mean, get_indices_feature_reprojection
from metrics.metrics_class import Metrics
import matplotlib.pyplot as plt 
import shutil
from tqdm import tqdm 

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def color_pcd_by_labels(pcd, labels,colors=None,gt_labels=None,semantics=False):
    
    if colors == None : 
        colors = generate_random_colors(2000)
    pcd_colored = copy.deepcopy(pcd)
    pcd_colors = np.zeros(np.asarray(pcd.points).shape)
    if gt_labels is None :
    	unique_labels = list(np.unique(labels)) 
    else: 
        unique_labels = list(np.unique(gt_labels))
    
    background_color = np.array([0,0,0])
    #for i in range(len(pcd_colored.points)):
    for i in unique_labels:
        if i == -1 : 
            continue
        idcs = np.where(labels == i)
        idcs = idcs[0]
        if i == 0 and semantics == False : 
            pcd_colors[idcs] = background_color
        else : 
            pcd_colors[idcs] = np.array(colors[unique_labels.index(i)])
        
    if semantics : 
        pcd_colored.colors = o3d.utility.Vector3dVector(pcd_colors)
    else : 
        pcd_colored.colors = o3d.utility.Vector3dVector(pcd_colors/255)
    return pcd_colored
    
def divide_indices_into_chunks(max_index, chunk_size=1000):
    chunks = []
    for start in range(0, max_index, chunk_size):
        end = min(start + chunk_size, max_index)
        chunks.append((start, end))
    return chunks



Here we define the dataset depending on kitti sequence!

In [3]:
DATASET_PATH = os.path.join('/media/cedric/Datasets2/semantic_kitti/')
end_inds = {0:4541,1:1100,2:4661,3:800,4:271,5:2761,6:1101,7:1100,8:4071,9:1591,10:1201}
SEQUENCE_NUM = 0
old = False  

minor_voxel_size = 0.05
major_voxel_size = 0.35
chunk_size = np.array([25, 25, 25]) #meters
overlap = 3 #meters
ground_segmentation_method = 'patchwork' 
NCUT_ground = False

out_chunks = 'pcd_preprocessed/output_chunks/'

out_folder_ncuts = out_chunks + 'test_data' + str(SEQUENCE_NUM) + '/'
if os.path.exists(out_folder_ncuts) == False :
        os.makedirs(out_folder_ncuts)

dataset = create_kitti_odometry_dataset(DATASET_PATH,SEQUENCE_NUM,ncuts_mode=True)

out_folder = 'pcd_preprocessed/'
if os.path.exists(out_folder) == False : 
        os.makedirs(out_folder)



out_kitti_instance = out_chunks + 'out_kitti_instance' + str(SEQUENCE_NUM) + '/'
        
if os.path.exists(out_kitti_instance) == False : 
        os.makedirs(out_kitti_instance)
 
alpha = 1.0
theta = 0.5
colors = generate_random_colors_map(600)
beta = 0.0
gamma = 0.1
proximity_threshold = 1.0
tarl_norm = False
ncuts_threshold = 0.005
        
out_name = 'tarl_only' + str(SEQUENCE_NUM) 


Now we aggregate a large point cloud based on (ind_start, ind_end)
## This cell can be ignored after first run as outputs are stored 

This cell can be ignored after first run as outputs are stored 

In [6]:
##load data if already stored 



chunks_idcs = divide_indices_into_chunks(len(dataset)) 

for cur_idx, cidcs in enumerate(chunks_idcs[4:]): 
        ind_start, ind_end = cidcs[0], cidcs[1]
        cur_idx = int(ind_start/1000) 
        print('cur idx',cur_idx)
        print("ind start",ind_start)
        print('ind end',ind_end)
        
        if os.path.exists(f'{out_folder}all_poses_{SEQUENCE_NUM}_{cur_idx}.npz') == False:
                process_and_save_point_clouds(dataset,ind_start,ind_end,minor_voxel_size=minor_voxel_size,
                                        major_voxel_size=major_voxel_size,icp=False,
                                        out_folder=out_folder,sequence_num=SEQUENCE_NUM,
                                        ground_segmentation_method=ground_segmentation_method,cur_idx=cur_idx)
        
        if os.path.exists(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_{cur_idx}.pcd') == False:
                print('load and downsample')
                pcd_ground_minor, pcd_nonground_minor,\
                        all_poses, T_pcd, first_position,kitti_labels = load_and_downsample_point_clouds(out_folder,SEQUENCE_NUM,minor_voxel_size,\
                                                                                ground_mode=ground_segmentation_method,cur_idx=cur_idx)
                #o3d.visualization.draw_geometries([color_pcd_by_labels(pcd_nonground_minor,kitti_labels['seg_nonground'])])
                o3d.io.write_point_cloud(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_{cur_idx}.pcd', pcd_ground_minor, write_ascii=False, compressed=False, print_progress=False)
                o3d.io.write_point_cloud(f'{out_folder}pcd_nonground_minor{SEQUENCE_NUM}_{cur_idx}.pcd', pcd_nonground_minor, write_ascii=False, compressed=False, print_progress=False)
                np.savez(f'{out_folder}kitti_labels_preprocessed{SEQUENCE_NUM}_{cur_idx}.npz',
                                                        instance_nonground=kitti_labels['instance_nonground'],
                                                        instance_ground=kitti_labels['instance_ground'],
                                                        seg_ground = kitti_labels['seg_ground'],
                                                        seg_nonground=kitti_labels['seg_nonground']
                                                        )
        
        pcd_ground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_{cur_idx}.pcd')
        pcd_nonground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_nonground_minor{SEQUENCE_NUM}_{cur_idx}.pcd')
        print(pcd_ground_minor)
        kitti_labels_orig = {}
        with np.load(f'{out_folder}kitti_labels_preprocessed{SEQUENCE_NUM}_{cur_idx}.npz') as data :
                kitti_labels_orig['instance_ground'] = data['instance_ground']
                kitti_labels_orig['instance_nonground'] = data['instance_nonground']
                kitti_labels_orig['seg_nonground'] = data['seg_nonground']
                kitti_labels_orig['seg_ground'] = data['seg_ground']
        
                
        
        with np.load(f'{out_folder}all_poses_{SEQUENCE_NUM}_{cur_idx}.npz') as data:
                all_poses = data['all_poses']
                T_pcd = data['T_pcd']
                first_position = T_pcd[:3, 3]
                
        if os.path.exists(f'{out_folder}subsampled_data{str(SEQUENCE_NUM)}_{cur_idx}.npz') == False : 
                print(f'{out_folder}subsampled_data{str(SEQUENCE_NUM)}_{cur_idx}.npz')
                poses, positions, \
        	sampled_indices_local, sampled_indices_global = subsample_and_extract_positions(all_poses,ind_start=ind_start,sequence_num=SEQUENCE_NUM,
        	                                                                        out_folder=out_folder,cur_idx=cur_idx)
        
        with np.load(f'{out_folder}subsampled_data{SEQUENCE_NUM}_{cur_idx}.npz') as data:
                poses=data['poses']
                positions=data['positions']
                sampled_indices_local = data['sampled_indices_local']
                sampled_indices_global=data['sampled_indices_global']
        
        print("chunk and downsample")
        pcd_nonground_chunks, pcd_ground_chunks,\
        pcd_nonground_chunks_major_downsampling, pcd_ground_chunks_major_downsampling, \
        indices,indices_ground, center_positions, \
        center_ids, chunk_bounds, kitti_labels, obbs = chunk_and_downsample_point_clouds(dataset,pcd_nonground_minor, pcd_ground_minor, T_pcd, positions, 
                                                                    first_position, sampled_indices_global, chunk_size=chunk_size, 
                                                                    overlap=overlap, major_voxel_size=major_voxel_size,kitti_labels=kitti_labels_orig)
                                                                    
        
        
        # Generate 30 different colors
        
        
        COLORS = generate_random_colors(400)
        COLORS = [(col[0]/255.,col[1]/255.,col[2]/255.) for col in COLORS]
        

                
        

        
        limit = -1 ##use this for experiments to run limit chunks numberss
        
        
        patchwise_indices = indices_per_patch(T_pcd, center_positions, positions, first_position, sampled_indices_global, chunk_size)
        out_data = []
        semantics = np.hstack((kitti_labels_orig['seg_nonground'].reshape(-1,),kitti_labels_orig['seg_ground'].reshape(-1,)))
        
        instances = np.hstack((kitti_labels_orig['instance_nonground'].reshape(-1,),kitti_labels_orig['instance_ground'].reshape(-1,)))
                        
        merge_pcd = o3d.geometry.PointCloud()    
        
        out_kitti_instance_cur = out_kitti_instance + str(cur_idx) + '/'
        out_folder_ncuts_cur = out_folder_ncuts + str(cur_idx) + '/'
        if os.path.exists(out_kitti_instance_cur) == False : 
                #shutil.rmtree(out_kitti_instance_cur)
                os.makedirs(out_kitti_instance_cur)
                
        if os.path.exists(out_folder_ncuts_cur) == False : 
                os.makedirs(out_folder_ncuts_cur)
        
        print("total sequence number",len(center_ids))
        for sequence in tqdm(range(30,len(center_ids))):
                        print('center',center_ids[sequence])
                        merged_chunk,file_name, pcd_chunk, pcd_chunk_ground,inliers, inliers_ground = ncuts_chunk(dataset,list(indices),pcd_nonground_chunks,pcd_ground_chunks,
                                pcd_nonground_chunks_major_downsampling,
                                pcd_nonground_minor,T_pcd,center_positions,center_ids,
                                positions,first_position,list(sampled_indices_global),
                                chunk_size=chunk_size,major_voxel_size=major_voxel_size,
                                alpha=alpha,beta=beta,gamma=gamma,theta=theta,
                                proximity_threshold=proximity_threshold,
                                out_folder=out_folder_ncuts,ground_mode=False,sequence=sequence,
                                patchwise_indices=patchwise_indices,ncuts_threshold=ncuts_threshold,obb=obbs[sequence])
                        
                                
                        
                        
                        
                        seg_ground = kitti_labels['ground']['semantic'][sequence][inliers][inliers_ground]
                        inst_ground = kitti_labels['ground']['instance'][sequence][inliers][inliers_ground]
                        
                        file_name = str(center_ids[sequence]).zfill(6) + '.pcd'
        
                        kitti_chunk = color_pcd_by_labels(pcd_chunk,kitti_labels['nonground']['semantic'][sequence].reshape(-1,),
                                                colors=COLORS,gt_labels=semantics,semantics=True
                                                )
                        
                        kitti_chunk_instance = color_pcd_by_labels(pcd_chunk,kitti_labels['nonground']['instance'][sequence].reshape(-1,),
                                                colors=colors,gt_labels=instances)
                                                
                        kitti_chunk_instance_ground = color_pcd_by_labels(pcd_chunk_ground,inst_ground.reshape(-1,),
                                                colors=colors,gt_labels=instances)
                                                
                        kitti_chunk_semantic_ground = color_pcd_by_labels(pcd_chunk_ground,seg_ground.reshape(-1,),
                                                colors=COLORS,gt_labels=semantics,semantics=True)
                        
                        #o3d.visualization.draw_geometries([kitti_chunk_instance + kitti_chunk_instance_ground])
                        #o3d.visualization.draw_geometries([pcd_chunk + pcd_chunk_ground])
                        #semantic_labels = np.hstack(( kitti_labels['nonground']['semantic'][sequence].reshape(-1,),seg_ground.reshape(-1,)))
                        #np.savez(out_kitti_semantic + file_name.split('.')[0] + '.npz',labels=semantic_labels)
                        
                        o3d.io.write_point_cloud(out_folder_ncuts_cur + file_name, pcd_chunk + pcd_chunk_ground , write_ascii=False, compressed=False, print_progress=False)
                        print('write instance file',out_kitti_instance_cur + file_name)
                        #o3d.io.write_point_cloud(out_kitti_instance_cur + file_name, kitti_chunk_instance + kitti_chunk_instance_ground, write_ascii=False, compressed=False, print_progress=False)
                        #o3d.io.write_point_cloud(out_kitti_semantic + file_name, kitti_chunk + kitti_chunk_semantic_ground, write_ascii=False, compressed=False, print_progress=False)
                        
                

cur idx 4
ind start 4000
ind end 4661
PointCloud with 9588558 points.
chunk and downsample


100%|██████████| 631/631 [00:21<00:00, 29.40it/s]


Downsampled from (295054, 3) to (5556, 3) points (non-ground)
Downsampled from (276045, 3) to (4905, 3) points (ground)
Downsampled from (366729, 3) to (6955, 3) points (non-ground)
Downsampled from (331133, 3) to (6079, 3) points (ground)
Downsampled from (360301, 3) to (6248, 3) points (non-ground)
Downsampled from (406894, 3) to (6523, 3) points (ground)
Downsampled from (476949, 3) to (7853, 3) points (non-ground)
Downsampled from (340785, 3) to (5662, 3) points (ground)
Downsampled from (354459, 3) to (6041, 3) points (non-ground)
Downsampled from (329276, 3) to (5788, 3) points (ground)
Downsampled from (403509, 3) to (6075, 3) points (non-ground)
Downsampled from (312004, 3) to (5091, 3) points (ground)
Downsampled from (482147, 3) to (6593, 3) points (non-ground)
Downsampled from (359209, 3) to (5257, 3) points (ground)
Downsampled from (270854, 3) to (3689, 3) points (non-ground)
Downsampled from (397530, 3) to (6036, 3) points (ground)
Downsampled from (342510, 3) to (5104, 3

  0%|          | 0/4 [00:00<?, ?it/s]

center 4595
Start of sequence 30
4327 points in downsampled chunk (major)
label shape (376, 1241)
Adjacency Matrix built
0 isolated points removed
Start of normalized Cuts
--------------
graph construction  653.1563885211945  s
There are 9 cut regions
NCuts took  2.498717784881592  s
Ratio of points in top 3 groups: 0.7938525537323781
Dino load  656.7840938568115  s
Ncuts percentage  0.0
Construction  99.0


...

write instance file pcd_preprocessed/output_chunks/out_kitti_instance2/4/004612.pcd
center 4629
Start of sequence 32
4471 points in downsampled chunk (major)
label shape (376, 1241)
Adjacency Matrix built
0 isolated points removed
Start of normalized Cuts
--------------
graph construction  797.8358774185181  s
There are 10 cut regions
NCuts took  4.17555570602417  s
Ratio of points in top 3 groups: 0.9689107582196377
Dino load  802.9667413234711  s
Ncuts percentage  1.0
Construction  99.0


 75%|███████▌  | 3/4 [34:18<11:44, 704.92s/it]

write instance file pcd_preprocessed/output_chunks/out_kitti_instance2/4/004629.pcd
center 4646
Start of sequence 33
4886 points in downsampled chunk (major)
label shape (376, 1241)
Adjacency Matrix built
0 isolated points removed
Start of normalized Cuts
--------------
graph construction  547.5776963233948  s
There are 10 cut regions
NCuts took  4.115172386169434  s
Ratio of points in top 3 groups: 0.9928366762177651
Dino load  553.0466420650482  s
Ncuts percentage  1.0
Construction  99.0


100%|██████████| 4/4 [43:32<00:00, 653.12s/it]

write instance file pcd_preprocessed/output_chunks/out_kitti_instance2/4/004646.pcd


Now we subsample the poses based on a voxel_size

Now we can split the point cloud into chunks based on a tbd chunk_size

In [7]:



for i in range(len(divide_indices_into_chunks(len(dataset)))) : 
	
	out_kitti_instance_cur = out_kitti_instance + str(i) + '/'
	out_folder_ncuts_cur = out_folder_ncuts + str(i) + '/'
	
	point_clouds = get_merge_pcds(out_folder_ncuts_cur)
	if len(point_clouds) == 0 : 
		continue
	point_clouds_kitti_instances = get_merge_pcds(out_kitti_instance_cur)
	merge_pred = merge_chunks_unite_instances2(point_clouds)
	#merge_kitti_instance = merge_unite_gt(point_clouds_kitti_instances)
	
	#o3d.io.write_point_cloud(out_folder + "merge_part_kitti_semantic7.pcd", merge_kitti_semantic, write_ascii=False, compressed=False, print_progress=False)
	o3d.io.write_point_cloud(out_folder_ncuts + str(SEQUENCE_NUM) + "_" + str(i) +  "_pred_all.pcd", merge_pred, write_ascii=False, compressed=False, print_progress=False)
	#o3d.io.write_point_cloud(out_kitti_instance + "merge_part_kitti_instance" + str(SEQUENCE_NUM) + "_" + str(i) +  ".pcd", merge_kitti_instance, write_ascii=False, compressed=False, print_progress=False)


['000015.pcd', '000031.pcd', '000051.pcd', '000075.pcd', '000098.pcd', '000119.pcd', '000139.pcd', '000157.pcd', '000175.pcd', '000194.pcd', '000212.pcd', '000230.pcd', '000248.pcd', '000266.pcd', '000285.pcd', '000304.pcd', '000322.pcd', '000339.pcd', '000356.pcd', '000373.pcd', '000391.pcd', '000410.pcd', '000430.pcd', '000451.pcd', '000472.pcd', '000495.pcd', '000523.pcd', '000546.pcd', '000569.pcd', '000589.pcd', '000607.pcd', '000625.pcd', '000643.pcd', '000661.pcd', '000679.pcd', '000697.pcd', '000714.pcd', '000731.pcd', '000749.pcd', '000768.pcd', '000786.pcd', '000803.pcd', '000820.pcd', '000837.pcd', '000855.pcd', '000876.pcd', '000909.pcd', '000942.pcd', '000966.pcd', '000990.pcd']
[]


100%|██████████| 49/49 [05:36<00:00,  6.86s/it]


['001042.pcd', '001073.pcd', '001102.pcd', '001129.pcd', '001161.pcd', '001191.pcd', '001215.pcd', '001238.pcd', '001260.pcd', '001286.pcd', '001324.pcd', '001353.pcd', '001377.pcd', '001402.pcd', '001426.pcd', '001449.pcd', '001471.pcd', '001491.pcd', '001510.pcd', '001528.pcd', '001546.pcd', '001564.pcd', '001583.pcd', '001603.pcd', '001624.pcd', '001646.pcd', '001668.pcd', '001690.pcd', '001710.pcd', '001729.pcd', '001753.pcd', '001804.pcd', '001829.pcd', '001848.pcd', '001865.pcd', '001880.pcd', '001896.pcd', '001912.pcd', '001927.pcd', '001942.pcd', '001958.pcd', '001980.pcd']
[]


100%|██████████| 41/41 [04:44<00:00,  6.94s/it]


['002033.pcd', '002055.pcd', '002075.pcd', '002093.pcd', '002111.pcd', '002128.pcd', '002145.pcd', '002163.pcd', '002181.pcd', '002199.pcd', '002216.pcd', '002233.pcd', '002250.pcd', '002267.pcd', '002284.pcd', '002301.pcd', '002318.pcd', '002335.pcd', '002353.pcd', '002371.pcd', '002390.pcd', '002411.pcd', '002435.pcd', '002459.pcd', '002480.pcd', '002499.pcd', '002518.pcd', '002541.pcd', '002576.pcd', '002601.pcd', '002621.pcd', '002649.pcd', '002678.pcd', '002700.pcd', '002720.pcd', '002741.pcd', '002763.pcd', '002783.pcd', '002802.pcd', '002821.pcd', '002840.pcd', '002859.pcd', '002878.pcd', '002897.pcd', '002916.pcd', '002935.pcd', '002954.pcd', '002972.pcd', '002990.pcd']
[]


100%|██████████| 48/48 [06:22<00:00,  7.98s/it]


['003018.pcd', '003036.pcd', '003055.pcd', '003074.pcd', '003094.pcd', '003115.pcd', '003135.pcd', '003153.pcd', '003171.pcd', '003189.pcd', '003207.pcd', '003226.pcd', '003245.pcd', '003266.pcd', '003289.pcd', '003330.pcd', '003361.pcd', '003385.pcd', '003418.pcd', '003443.pcd', '003465.pcd', '003490.pcd', '003514.pcd', '003534.pcd', '003553.pcd', '003572.pcd', '003589.pcd', '003609.pcd', '003638.pcd', '003661.pcd', '003680.pcd', '003701.pcd', '003726.pcd', '003748.pcd', '003769.pcd', '003793.pcd', '003819.pcd', '003840.pcd', '003858.pcd', '003876.pcd', '003894.pcd', '003911.pcd', '003928.pcd', '003946.pcd', '003963.pcd', '003980.pcd', '003998.pcd']
[]


100%|██████████| 46/46 [05:39<00:00,  7.39s/it]


['004018.pcd', '004037.pcd', '004065.pcd', '004094.pcd', '004116.pcd', '004136.pcd', '004156.pcd', '004187.pcd', '004215.pcd', '004234.pcd', '004252.pcd', '004269.pcd', '004286.pcd', '004302.pcd', '004318.pcd', '004334.pcd', '004350.pcd', '004367.pcd', '004385.pcd', '004402.pcd', '004420.pcd', '004438.pcd', '004457.pcd', '004473.pcd', '004491.pcd', '004508.pcd', '004525.pcd', '004542.pcd', '004560.pcd', '004578.pcd', '004595.pcd', '004612.pcd', '004629.pcd', '004646.pcd']
[]


100%|██████████| 33/33 [03:08<00:00,  5.72s/it]


In [8]:
metrics_ncuts = Metrics('ncuts')

for i in range(len(divide_indices_into_chunks(len(dataset)))) : 

		if os.path.exists(out_folder_ncuts + str(SEQUENCE_NUM) + "_" + str(i) +  "_pred_all.pcd") == False : 
			continue
		all_pred = o3d.io.read_point_cloud(out_folder_ncuts + str(SEQUENCE_NUM) + "_" + str(i) +  "_pred_all.pcd")
		cur_gt = o3d.io.read_point_cloud(out_kitti_instance + "merge_part_kitti_instance" + str(SEQUENCE_NUM) + "_" + str(i) +  ".pcd")
			
		unique_colors, labels_ncuts_all = np.unique(np.asarray(all_pred.colors), axis=0, return_inverse=True)
		unique_colors, labels_kitti = np.unique(np.asarray(cur_gt.colors),axis=0, return_inverse=True)
		
		labels_ncuts = remove_semantics(labels_kitti,labels_ncuts_all)
		metrics_ncuts.add_stats(labels_ncuts_all,labels_ncuts,labels_kitti)
	


metrics_ncuts.compute_stats_final()


start
Metrics for file ncuts


Processing: 100%|██████████| 269/269 [00:29<00:00,  9.05it/s]


Precison @ 0.5 0.8883248730964467
Recall @ 0.5 0.7322175732217573
F Score @ 0.5 0.8027522935779815
Mean @ 0.5 0.9080574000428899
Panoptic @ 0.5 0.7289451605848886
S_assoc score 0.6966849185625403
AP @ 0.25 73.283
AP @ 0.5 63.111
AP @ [0.5:0.95] 47.249
